In [1]:
#------------------------------------------------------------------------------------------------
# 03. Corpus-based analysis and synthesis
#
# This notebook demonstrates how to:
#  - extract features from a pre-recorded corpus of audio (here, a large audio file)
#  - perform dimensionality reduction
#  - transform feature arrays into a pandas dataframe
#  - create interactive plots to play back grains in latent feature space
#  - use audio-rate modulation to modulate playback of grains
#  - use timeline-based sequencing to trigger playback of grains
#
# TODO: Different segmentation methods
#
# Requirements:
#   pip3 install numpy librosa scikit-learn ipython matplotlib altair pandas anywidget isobar
#------------------------------------------------------------------------------------------------

import librosa
import warnings
import numpy as np
import pandas as pd
import altair as alt
import sklearn.cluster
import IPython.display
import sklearn.decomposition
import sklearn.preprocessing
import matplotlib.pyplot as plt


# Hide numerical warnings and permit large datasets
warnings.filterwarnings('ignore')
alt.data_transformers.disable_max_rows()

from signalflow import *
graph = AudioGraph()

Output device: MacBook Pro Speakers (44100Hz, buffer size 512 samples, 2 channels)


In [2]:
import os
import torchaudio
import random as r
from tqdm import tqdm

path = "/Volumes/T7_Touch/ocd/generation/data_output/staging/rendered_audio/"
files = os.listdir(path)
r.shuffle(files)
fs = []
embeddings = []
audio_list = []

for file in tqdm(files[:1000]):
    if ".flac" in file:
        try:
            full_path = path + file
            audio, _ = torchaudio.load(full_path)
            audio_list.append( audio.squeeze(0).numpy())

            
            fs.append(full_path)
        except Exception as e:
            print("error", e)

100%|██████████████████████████████████████| 1000/1000 [00:06<00:00, 161.57it/s]


In [3]:
buffer = np.concatenate(audio_list, axis=0)


In [4]:
#--------------------------------------------------------------------------------
# Set global variables: FFT size, hop size, sample rate
#--------------------------------------------------------------------------------
fft_size = 16384
# fft_size = 4096
hop_size = fft_size // 2
# sample_rate = buffer.sample_rate
sample_rate = 44100

#--------------------------------------------------------------------------------
# Extract MFCC, and rescale to zero-mean, unit-variance.
# FluCoMa has a nice interactive explainer that gives some intuition:
# https://learn.flucoma.org/reference/mfcc/
#--------------------------------------------------------------------------------
# X = librosa.feature.mfcc(y=buffer.data[0], sr=sample_rate, n_fft=fft_size, hop_length=hop_size, n_mfcc=20)
X = librosa.feature.mfcc(y=buffer, sr=sample_rate, n_fft=fft_size, hop_length=hop_size, n_mfcc=20)
X = sklearn.preprocessing.scale(X)
X = X.T
print("MFCC coefficient shape: %s" % str(X.shape))
print(np.round(X[:8,:8], 4))

MFCC coefficient shape: (26917, 20)
[[-2.9729  3.162   0.4507 -0.4222 -0.1877 -0.1119 -0.1236 -0.1699]
 [-0.7176  4.173  -0.7612 -0.8261  0.0896 -0.2896 -0.289  -0.2838]
 [-0.2065  4.1702 -1.0852 -0.7815  0.0539 -0.3509 -0.2437 -0.291 ]
 [ 0.1406  4.1059 -1.3023 -0.7566 -0.1064 -0.3528 -0.1043 -0.2919]
 [ 0.1922  4.0749 -1.4514 -0.8108 -0.0799 -0.4151 -0.1616 -0.3133]
 [ 0.3341  4.0822 -1.4368 -0.6787 -0.1265 -0.4006 -0.1271 -0.3663]
 [ 0.3226  4.1156 -1.3874 -0.5511 -0.0314 -0.3809 -0.0883 -0.3145]
 [ 0.2045  4.1606 -1.1906 -0.714  -0.0363 -0.3259 -0.0795 -0.2811]]


In [5]:
#--------------------------------------------------------------------------------
# Perform Principal Component Analysis (PCA) to reduce the dimensionality of
# each input MFCC frame, and extract various manually-specified features.
#--------------------------------------------------------------------------------
model = sklearn.decomposition.PCA(n_components=3, whiten=True)
model.fit(X)
Y = model.transform(X)
print("Y shape: %s" % str(Y.shape))

#--------------------------------------------------------------------------------
# Create data series, containing per-segment properties which are later needed
# for display and playback:
#  - ordinal index
#  - timestamp (in seconds)
#  - duration (the same for every block, as we're using identically-sized blocks)
#--------------------------------------------------------------------------------
index = np.arange(len(Y))
timestamp = index * hop_size / sample_rate
duration = fft_size / sample_rate
duration_array = np.array([duration] * len(Y))

def floats_to_ordinals(floats):
    sorted_indices = np.argsort(floats)
    positions = np.argsort(sorted_indices)
    return positions.tolist()

#--------------------------------------------------------------------------------
# Manually extract a few features to add to the data frame:
#  - spectral centroid
#  - spectral flatness
#  - k-means cluster
#--------------------------------------------------------------------------------
centroid = librosa.feature.spectral_centroid(y=buffer, sr=44100, n_fft=fft_size, hop_length=hop_size)[0]
flatness = librosa.feature.spectral_flatness(y=buffer, n_fft=fft_size, hop_length=hop_size)[0]
flatness = floats_to_ordinals(flatness)
kmeans = sklearn.cluster.KMeans(n_clusters=4)
labels = kmeans.fit_predict(Y)

#--------------------------------------------------------------------------------
# Aggregate all features into a pandas DataFrame, with columns for each feature
# and a row for each segment
#--------------------------------------------------------------------------------
df = pd.DataFrame({
    "a": Y[:,0],
    "b": Y[:,1],
    "c": Y[:,2],
    "centroid": centroid,
    "flatness": flatness,
    "index": index,
    "timestamp": timestamp,
    "duration": duration_array,
    "cluster": labels,
})
df

Y shape: (26917, 3)


,a,b,c,centroid,flatness,index,timestamp,duration,cluster
0,0.111015,-0.778469,-0.916196,1449.181696,11156,0,0.000000,0.371519,3
1,1.403363,0.344357,-0.538042,1512.648117,8440,1,0.185760,0.371519,2
2,1.611404,0.751074,-0.073680,1600.866441,9345,2,0.371519,0.371519,1
3,1.730186,1.070612,0.299991,1750.350414,11863,3,0.557279,0.371519,1
4,1.745089,1.224831,0.256057,1863.748744,13142,4,0.743039,0.371519,1
...,...,...,...,...,...,...,...,...,...
26912,-1.045595,-0.288068,-0.059153,1470.725652,6896,26912,4999.163356,0.371519,0
26913,-1.347616,-0.003363,0.635716,1300.324130,13699,26913,4999.349116,0.371519,0
26914,-1.438623,0.235462,0.775523,1612.773538,20171,26914,4999.534875,0.371519,0
26915,-1.461206,0.292682,0.813081,2054.116129,24089,26915,4999.720635,0.371519,0


In [6]:
buffer = Buffer(buffer)

In [7]:
#--------------------------------------------------------------------------------
# Altair scatter plot displaying each grain's location within feature space,
# with a point-based selector to trigger playback of grains
#--------------------------------------------------------------------------------


chart = alt.Chart(df, width=800, height=500)
chart = chart.mark_circle(size=40)
chart = chart.encode(x=alt.X("a"),
                     y=alt.Y("b"),
                     color=alt.Color('timestamp').scale(scheme="plasma"),
                     tooltip=["index", "timestamp"])

selector = alt.selection_point(name="point",
                               on='mouseover',
                               nearest=True)
chart = chart.add_params(selector)

#--------------------------------------------------------------------------------
# SegmentedGranulator plays back a segment of the input file when triggered
#--------------------------------------------------------------------------------
granulator = SegmentedGranulator(buffer,
                                 df.timestamp,
                                 df.duration)
# granulator.set_buffer("envelope", EnvelopeBuffer("hanning"))
granulator.set_buffer("envelope", EnvelopeBuffer("linear-decay"))
#granulator.set_buffer("envelope", EnvelopeBuffer("triangle"))
attenuated = granulator * 0.75
attenuated.play()

#--------------------------------------------------------------------------------
# Altair callback to trigger a grain on hover
#--------------------------------------------------------------------------------
def on_select(change):
    value = change["new"].value
    if value:
        index = value[0]
        granulator.trigger("trigger", index)
        granulator.index = index

#--------------------------------------------------------------------------------
# Add Jupyter interactivity
#--------------------------------------------------------------------------------
jchart = alt.JupyterChart(chart)
jchart.selections.observe(on_select, ["point"])
jchart

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'data': {'name': 'da…

In [13]:
#--------------------------------------------------------------------------------
# This example demonstrates playing features in real-time, using SignalFlow
# LFO objects to modulate the X/Y position in feature space.
#
# The NearestNeighbour node performs a search for the nearest datapoint to
# the specified `target` coordinate.
# 
# The signalflow_analysis library contains the AudioFeatureBuffer object,
# which encodes N-dimensional frame-wise feature properties. AudioFeatureBuffer
# is a subclass of the generic "Buffer" signalflow class.
#
#  - Each channel of the buffer corresponds to a feature
#  - Each sample in the buffer corresponds to a segment (block) of the input
#--------------------------------------------------------------------------------

from signalflow_analysis import *

xpos = SineLFO(0.5, -1, 1)
ypos = SineLFO(0.71, -1, 1)
feature_buffer = AudioFeatureBuffer([df.a, df.b])
nearest_index = NearestNeighbour(feature_buffer, target=[xpos, ypos])
player = SegmentedGranulator(buffer=buffer,
                             onset_times=df.timestamp,
                             durations=df.duration,
                             index=nearest_index,
                             clock=RandomImpulse(5))
attenuated = player * 0.5
attenuated.play()

In [14]:
#--------------------------------------------------------------------------------
# Modulate playback parameters interactively.
#--------------------------------------------------------------------------------
player.clock = RandomImpulse(20)
nearest_index.target = [xpos, ypos]

In [27]:
#--------------------------------------------------------------------------------
# Example using the isobar sister library for sequencing note events.
#--------------------------------------------------------------------------------

from isobar import *

class NearestNeighbourTrigger (Patch):
    #--------------------------------------------------------------------------------
    # This patch encapsulates a granulator that plays back grains selected via
    # a feature buffer, when triggered.
    #--------------------------------------------------------------------------------
    def __init__(self):
        super().__init__()
        x = self.add_input("centroid")
        y = self.add_input("flatness")
        feature_buffer = AudioFeatureBuffer([df.centroid, df.flatness])
        nn = NearestNeighbour(feature_buffer, [x, y])
        granulator = SegmentedGranulator(buffer,
                                         df.timestamp,
                                         df.duration * 0.2,
                                         index=nn)
        # how to create envelope buffer from segments / shape?
        granulator.set_buffer("envelope", EnvelopeBuffer("linear-decay"))
        delay = AllpassDelay(granulator, 0.1, feedback=0.9)
        output = granulator + delay * 0.5
        self.set_output(output)
        self.set_trigger_node(granulator)

#--------------------------------------------------------------------------------
# Create the patch and connect it to the graph.
#--------------------------------------------------------------------------------
nnpatch = NearestNeighbourTrigger()
nnpatch.play()

#--------------------------------------------------------------------------------
# Create a 150bpm timeline.
#--------------------------------------------------------------------------------
timeline = Timeline(100, output_device=SignalFlowOutputDevice(graph=graph))
timeline.background()

In [33]:
timeline.schedule({
    "patch": nnpatch,
    "type": "trigger",
    "duration": 0.5,
    "params": {
        # "centroid": PSequence([9000, 200, 9000, 100, 7000]) + PWhite(-200, 1000),
        # "flatness": PSequence([100, 3000, 2000, 3000]) + PWhite(-500, 500)
        "centroid": PSequence([0 ]) + PWhite(-200, 1000),
        "flatness": PSequence([100, 3000, 2000]) + PWhite(-500, 500)
    }
}, name="track", replace=True)

In [26]:
 timeline.clear()
graph.clear()